## A quick benchmark for reads/writes

`LevelDB` at the moment is orders of magnitude faster than `LMDB`
The reason is that the bach edge writes are not implemented yet for `LMDB`

In [1]:
import os
data_path_google_plus= '/mnt/6c99ccdb-4d5e-443d-b361-8b5137623ef6/data/gplus/'
# 'featnames','edges', 'followers','circles'
file_suff = ['featnames','edges','followers','circles']
_d = {}
for fs in file_suff:
    _d_fs  = [os.path.join(data_path_google_plus, g) for g in os.listdir(data_path_google_plus) if fs in g]
    _d[fs] = _d_fs


In [2]:
import sys
sys.path.append('/home/charilaos/Workspace/pygraphdb/src')
import graphdb

In [3]:
from kvstores import LevelDBStore, LMDBStore
from graphdb import GraphDB, Node, Edge
from serializers import PickleSerializer
# _store = LMDBStore(path='/tmp/graph_lmdb_example_2', map_size=2**35)
_store = LevelDBStore('/tmp/leveldb_benchmark_write')
serializer = PickleSerializer()

# 2. Create the GraphDB 
graph_db = GraphDB(_store, serializer)

In [5]:
# follower relation:
from time import time
t0 = time()
num_writes = 0
node_writes = 0
edge_writes = 0

for followers_edge_list in _d['followers']:
    edge_list = []
    with open(followers_edge_list,'r') as f:
        _dd = f.read().split('\n')
        to_node = followers_edge_list.split('/')[-1].split('.followers')[0]
        for from_node in _dd:
            # edges.append((from_node, to_node))
            n1, n2 = Node(node_id = from_node), Node(node_id = to_node)
            e = Edge(source = n1.get_id, target = n2.get_id)
            graph_db.put_node(n1)
            graph_db.put_node(n2)
            node_writes += 2
            edge_list.append(e)
            # graph_db.put_edge(e)
            
        a = graph_db.put_edges_bulk(edge_list)
        edge_writes += len(edge_list)
        num_writes = edge_writes + node_writes

        if (num_writes % 1000) == 0:
            
            print(f'{num_writes} in {dt} (rate: {num_writes/dt})')
dt = time() - t0
print(f'{num_writes} writes in {dt}sec (write rate: {num_writes/dt} w/s or {dt/num_writes*1000} ms/w)')

338769 writes in 5.967495679855347sec (write rate: 56769.0398408821 w/s or 0.017615235395964053 ms/w)
